In [1]:
# Load the DSSM model and performs prediction

In [6]:
# Paths and constants
BatchSize = 256

db_path = './model/mediaDb.v1.sqlite'

db_folder = "val2014"

session_meta_path = './model/sess_stored_old_training.meta'
session_path = './model/sess_stored_old_training'


In [7]:
import numpy as np
import tensorflow as tf
import random

In [8]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
print(get_available_gpus())

['/device:GPU:0']


In [9]:
saver = tf.train.import_meta_graph(session_meta_path)

In [10]:
init = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(init)

In [11]:
saver.restore(sess, session_path)

INFO:tensorflow:Restoring parameters from ./model/sess_stored_old_training


In [12]:
graph = tf.get_default_graph()

# Get the output and input tensors. Prediction can be performed on exemplars by
y_exemplar_pred = graph.get_tensor_by_name("model_1/y_exemplar_pred:0")
exemplar_placeholder = graph.get_tensor_by_name("exemplar_placeholder:0")

#sample
#x = np.random.rand(10000, 1024)
#sess.run(y_exemplar_pred, feed_dict={exemplar_placeholder : x})

In [13]:
import sqlite3
import struct
conn = sqlite3.connect(db_path)

# Try adding DSSMEmbedding column to ItemEngineExemplar table. Will fail if it already exists, just ignore
from sqlite3 import OperationalError
try:
    cur = conn.cursor()
    cur.execute("ALTER TABLE ItemEngineExemplar ADD COLUMN ItemEngineExemplar_DSSMEmbedding BLOB")
except OperationalError:
    print ("Column probably exists already")
    
count = 0
cur = conn.cursor();
update_cur = conn.cursor();
cur.execute("SELECT Item_FileName as file, Folder_Path as path, ItemEngineExemplar_Exemplar as feature, ItemEngineExemplar_Id as id, ItemEngineExemplar_ItemId as itemId from ItemEngineExemplar, Folder, Item where ItemEngineExemplar_ItemId = Item_Id and Item_ParentFolderId = Folder_Id")

def process_batch(a, b):
    if len(b) <= 0:
        return
   
    outputs = sess.run(y_exemplar_pred, feed_dict={exemplar_placeholder : a})

    for i in range(0, len(b)):
        update_cur.execute("UPDATE ItemEngineExemplar SET ItemEngineExemplar_DSSMEmbedding = ? WHERE ItemEngineExemplar_Id=?", [outputs[i], b[i]]);

    conn.commit()
    
ids = []
inputs = []

print("Predicting and writing back to DB")
while True:
    count = count + 1
    
    if count % 500 == 0:
        print("Count so far: " + str(count))
        
    row = cur.fetchone()
    
    # Make sure we're not done, that we're in the correct folder and that there exists exemplar vector.
    if row == None:
        break
        
    if row[1].lower().find(db_folder) == -1:
        continue
        
    blob = row[2]
    
    if blob is None:
        continue
        
    data = np.array(struct.unpack('1024f', blob))
    
    ids.append(row[3])
    inputs.append(data)
    
    if len(ids) == BatchSize:
        process_batch(inputs, ids)
        ids = []
        inputs = []
        
# Process any remaining inputs
process_batch(inputs, ids)

print()
print("Closing database")
conn.close()
print("Done!")

Column probably exists already
Predicting and writing back to DB
Count so far: 500
Count so far: 1000
Count so far: 1500
Count so far: 2000
Count so far: 2500
Count so far: 3000
Count so far: 3500
Count so far: 4000
Count so far: 4500
Count so far: 5000
Count so far: 5500
Count so far: 6000
Count so far: 6500
Count so far: 7000
Count so far: 7500
Count so far: 8000
Count so far: 8500
Count so far: 9000
Count so far: 9500
Count so far: 10000
Count so far: 10500
Count so far: 11000
Count so far: 11500
Count so far: 12000
Count so far: 12500
Count so far: 13000
Count so far: 13500
Count so far: 14000
Count so far: 14500
Count so far: 15000
Count so far: 15500
Count so far: 16000
Count so far: 16500
Count so far: 17000
Count so far: 17500
Count so far: 18000
Count so far: 18500
Count so far: 19000
Count so far: 19500
Count so far: 20000
Count so far: 20500
Count so far: 21000
Count so far: 21500
Count so far: 22000
Count so far: 22500
Count so far: 23000
Count so far: 23500
Count so far: 2